In [18]:
!pip install flash-attn --no-build-isolation
!pip install git+https://github.com/TIGER-AI-Lab/Mantis.git

DEPRECATION: Loading egg at /home/g2/LLM-GGC-Upgrade/.env/lib/python3.12/site-packages/bitsandbytes-0.41.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 950.7 kB/s eta 0:00:000:00:01
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp312-cp312-linux_x86_64.whl size=187826523 sha256=7e0b07913d56782d0f3f2ee76bd39587557742628983f6ab9ec2527c99437476
  Stored in directory: /home/g2/.cache/pip/wheels/48/0f/98/166bf6b39dd0592924b22ce21b799fc9fd76c3a302c12ee359
Successfully built flash-attn
DEPRECATION: Loading egg at /home/g2/LLM-GGC-Upgrade/.env/lib/python3.12/site-packages/bitsandbytes-0.41.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()

import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"CUDA version string: {torch.version.cuda}")

PyTorch version: 2.6.0+cpu
CUDA available: False
CUDA device count: 0
CUDA version string: None


In [22]:
from mantis.models.mllava import chat_mllava
from PIL import Image
import flash_attn_2_cuda
import torch
import json
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model


ImportError: libc10_cuda.so: cannot open shared object file: No such file or directory

In [ ]:


model_path = "TIGER-Lab/MantisLLM-7B-V1"  # Base Mantis 7B model
# Load processor and model
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


In [ ]:

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    quantization_config=bnb_config,  # Apply quantization config here
    device_map="auto",
    _attn_implementation='eager',
    use_cache=False,
    torch_dtype=torch.bfloat16,
)

In [ ]:

# Load generation configuration
generation_config = GenerationConfig.from_pretrained(model_path, 'generation_config.json')

# %% Set up LoRA configuration for PEFT (Parameter Efficient Fine-Tuning)
peft_config = LoraConfig(
    r=8,  # Rank of the low-rank matrices for LoRA
    lora_alpha=32,  # Scaling factor for LoRA parameters
    target_modules=["q_proj", "v_proj", "k_proj"],  # LoRA applies to these modules
    lora_dropout=0.1,  # Dropout rate for LoRA
    bias="none",  # No bias for LoRA
    task_type="CAUSAL_LM",  # Task type, e.g., causal language modeling
)


In [ ]:

# Apply LoRA to the model for QLoRA
model = prepare_model_for_kbit_training(model)  # Prepare model for k-bit training (which is QLoRA)
model = get_peft_model(model, peft_config)  # Apply LoRA to the model

# Print trainable parameters (after LoRA is applied)
model.print_trainable_parameters()

# %% Load images
image1 = "image1.png"
image2 = "image2.png"
images = [Image.open(image1), Image.open(image2)]

# %% Load processor and model for chat generation
from mantis.models.mllava import MLlavaProcessor, LlavaForConditionalGeneration
processor = MLlavaProcessor.from_pretrained("TIGER-Lab/Mantis-8B-siglip-llama3")
model = LlavaForConditionalGeneration.from_pretrained(
    "TIGER-Lab/Mantis-8B-siglip-llama3",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation=None  # Using flash-attn if required
)

In [ ]:

# Set generation parameters
generation_kwargs = {
    "max_new_tokens": 24,
    "num_beams": 1,
    "do_sample": False,
}

# %% Chat with the model using the images
text = "Describe the difference of <image> and <image> as much as you can."
response, history = chat_mllava(text, images, model, processor, **generation_kwargs)
print("USER: ", text)
print("ASSISTANT: ", response)

# Continue chat with new text
text = "How many wallets are there in image 1 and image 2 respectively?"
response, history = chat_mllava(text, images, model, processor, history=history, **generation_kwargs)
print("USER: ", text)
print("ASSISTANT: ", response)

In [ ]:

# %% Load JSON file for processing
input_file = "/home/g2/ChartQA/ChartQA Dataset/test/test_human.json"  # Replace with your input JSON file path
output_file = "output.json"  # File to save generated answers

with open(input_file, "r") as f:
    data = json.load(f)

# %% Initialize Pix2Struct for chart QA
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

# Process each entry in the JSON file
previous_file_name = ""

# List to store results
results = []

# %% Iterate through each entry in the JSON file
for entry in data:
    file_name = entry["imgname"]
    file_path = "/home/g2/ChartQA/ChartQA Dataset/test/png/{}".format(file_name)
    question = entry["query"]
    expected_answer = entry["label"]

    print(type(question))

In [ ]:

    # Reload pipeline if the file name changes
    if file_name != previous_file_name:
        print(f"Loading new image: {file_name}")
        processor = Pix2StructProcessor.from_pretrained('google/deplot')
        model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')
        previous_file_name = file_name  # Update tracking variable

    # Open the image and verify it's valid
    try:
        image = Image.open(file_path)
        width, height = image.size  # Validate image dimensions (width and height must not be None)
        if width is None or height is None:
            print(f"Invalid image dimensions for {file_path}")
            continue  # Skip this image if dimensions are invalid
    except (IOError, SyntaxError) as e:
        print(f"Error loading image {file_path}: {e}")
        continue  # Skip this image and move to the next one

    # Process the image and query
    inputs = processor(images=image, text="Using the chart provided, answer the question: {}".format(question), return_tensors="pt")
    predictions = model.generate(**inputs, max_new_tokens=512)
    generated_answer = processor.decode(predictions[0], skip_special_tokens=True)

    print(question)
    print(generated_answer)

    # Store the result
    results.append({
        "imgname": file_name,
        "query": question,
        "label": expected_answer,
        "generated_answer": generated_answer
    })

In [ ]:

# %% Save results to output JSON
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {output_file}")
